In [1]:
import json
with open('db.json','r') as f:
    final_data = json.load(f)
dict_key = 'raw_content'
final_data[0]

{'_id': 1,
 'title': 'PDFBOX-1024-2.pdf',
 'path': './Final PDF\\PDFBOX-1024-2.pdf',
 'pages': 5,
 'content': 'Adobe Systems incorporated maintains royalty free, cross-platform documents. the content of this guide is furnished for informational use only. no part of the guide may be reproduced, stored in a retrieval system, or transmitted. no reference to companies or company logos in sample material is intended to refer to any actual organizations. do you want to know why i changed my copyright? click here. an alternative version is available. a total of 26 pages for free download at www.ebook.com ',
 'raw_content': 'AAT Adobe:PDF Adobe Batch Sequences Adobe Acrobat SDK Version 8.0 \n02 2006 Adobe Systems Incorporated. All rights reserved. Adobe Acrobate SDK 8.0 Batch Sequences for Microsoft Windows and Mac OS° Edition 1.0, October 2006 Ifthis guide is distributed with software that includes an end user agreement, this guide, as well as the software described in it, is furnished under 

In [2]:
# # train_data.to_excel('Temp Train df clean.xlsx', index=False)
import pandas as pd
train_data = pd.read_excel('Temp Train df raw.xlsx', names=['query', 'answer'])
train_data

,query,answer
0,who owns adobe acrobat,AAT Adobe:PDF Adobe Batch Sequences Adobe Acro...
1,what is adobe acrobat,AAT Adobe:PDF Adobe Batch Sequences Adobe Acro...
2,what is adobe batch sequences,AAT Adobe:PDF Adobe Batch Sequences Adobe Acro...
3,what is the mac adobe sdk version,AAT Adobe:PDF Adobe Batch Sequences Adobe Acro...
4,who owns adobe,AAT Adobe:PDF Adobe Batch Sequences Adobe Acro...
...,...,...
5655,what is the new kablink teaming code,DOCUMENTATION Developer Guide Kablink Teaming ...
5656,what does the gfdl license allow?,DOCUMENTATION Developer Guide Kablink Teaming ...
5657,who does not provide support for the novell pr...,DOCUMENTATION Developer Guide Kablink Teaming ...
5658,which license is the kablink package created u...,DOCUMENTATION Developer Guide Kablink Teaming ...


In [ ]:
from tqdm.notebook import tqdm_notebook
import pandas as pd
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration
dict_key = 'raw_content'
# tokenizer = T5Tokenizer.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')
tokenizer = T5Tokenizer.from_pretrained('BeIR/query-gen-msmarco-t5-base-v1')
q_model = T5ForConditionalGeneration.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1').to('cuda')
num_ques = 10
train_data = pd.DataFrame(columns=['query', 'answer'])
for i in tqdm_notebook(range(0,len(final_data),16), desc=f'Generating Qs'):
  sub_paragraphs = [x[dict_key] for x in final_data[i:i+16]]
  inputs = tokenizer(sub_paragraphs, return_tensors='pt', max_length=512, truncation=True, padding=True).to('cuda')
  outputs = q_model.generate(**inputs, max_length=100, do_sample=True, top_p=0.95, num_return_sequences=num_ques)
  outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  temp = pd.DataFrame({'query':outputs, 'answer':np.repeat(sub_paragraphs, num_ques)})
  train_data = pd.concat([train_data, temp], ignore_index=True)
  # if i> 15: break
train_data

In [3]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, SentenceTransformerTrainingArguments, SentenceTransformerModelCardData
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datasets import Dataset
# train_dataset = Dataset.from_pandas(train_data.iloc[:5000])
# test_dataset = Dataset.from_pandas(train_data.iloc[5000:])
train_dataset = Dataset.from_pandas(train_data)

train_dataset

c:\Users\RootAdmin\.virtualenvs\Digital_Doc-rmbKS4cs\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Dataset({
    features: ['query', 'answer'],
    num_rows: 5660
})

In [4]:
model = SentenceTransformer('sentence-transformers/msmarco-bert-base-dot-v5').to('cuda')
loss = MultipleNegativesRankingLoss(model)
num_epochs = 10
output_dir="sementic/bert-finetuned-raw",

args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir=output_dir,
    # Optional training parameters:
    
    num_train_epochs=num_epochs,
    per_device_train_batch_size=32,
    # per_device_eval_batch_size=16,
    warmup_ratio=0.1,
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    report_to = 'none'
    # run_name="mpnet-base-all-nli-triplet",  # Will be used in W&B if `wandb` is installed
)

c:\Users\RootAdmin\.virtualenvs\Digital_Doc-rmbKS4cs\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:

trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    loss=loss,
)
trainer.train()
model.save_pretrained(output_dir)

  0%|          | 0/1770 [00:00<?, ?it/s]

{'loss': 0.1883, 'grad_norm': 3.3100340366363525, 'learning_rate': 3.986189579409919e-05, 'epoch': 2.56}


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

{'loss': 0.0704, 'grad_norm': 0.0, 'learning_rate': 2.4168236032642812e-05, 'epoch': 5.13}
{'loss': 0.0493, 'grad_norm': 4.804586887359619, 'learning_rate': 8.47457627118644e-06, 'epoch': 7.56}
{'train_runtime': 563.9201, 'train_samples_per_second': 100.369, 'train_steps_per_second': 3.139, 'train_loss': 0.09237211572248383, 'epoch': 8.96}


TrainOutput(global_step=1770, training_loss=0.09237211572248383, metrics={'train_runtime': 563.9201, 'train_samples_per_second': 100.369, 'train_steps_per_second': 3.139, 'total_flos': 0.0, 'train_loss': 0.09237211572248383, 'epoch': 8.96045197740113})

In [8]:
import numpy as np
import faiss
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
doc_emb = model.encode([x[dict_key] for x in final_data])
index.add_with_ids(doc_emb, np.array([x['_id'] for x in final_data]))
faiss.write_index(index, './index/bert_raw_sementic.index')
index.ntotal

566

In [9]:
query = "how the characterization of diamond is done"
query_emb = model.encode([query])

top_k_arr = index.search(query_emb, 10)
print(top_k_arr)
a, b = top_k_arr[0][0], top_k_arr[1][0]
results = [final_data[b[i]-1] for i in range(2) if abs(a[0] - a[i]) < 10]
results

(array([[139.44911 , 110.36255 ,  83.13049 ,  71.892395,  66.716354,
         66.716354,  66.716354,  58.236713,  57.382412,  56.355743]],
      dtype=float32), array([[123, 393,  27, 206, 541,  87,  74, 363, 260,  29]], dtype=int64))


[{'_id': 123,
  'title': 'PDFBOX-2187-2.pdf',
  'path': './Final PDF\\PDFBOX-2187-2.pdf',
  'pages': 5,
  'content': 'secondary emitters have been developed for nea, and could be used to produce photocathodes. the injection devices would be pumped from a diamond into a thin stratum. primary electrons lose their energy in the first fraction of a micron of travel. the diamond acts as a vacuum barrier as well as an amplifier. a diamond emits secondary electrons to a vacuum where they travel to a distant point.',
  'raw_content': 'Electron Amplification in Diamond J. Smedley, I. Ben-Zvi, A. Burrill, X. Chang, J. Grimes, T. Rao, Z. Segalov, and Q. Wu\' Brookhaven National Laboratory, Upton NY, USA, IBNL and Indiana University Abstract. We report on recent progress toward development of secondary emission "amplifiers" for photocathodes. Secondary emission gain of over 300 has been achieved in transmission mode and emission mode for a variety of diamond samples. Techniques of sample preparati